In [1]:
module Q

using InteractiveUtils: subtypes

abstract type Creature end

Base.@kwdef mutable struct _Common
    hp::Int = 200
    atk::Int = 100
end

Base.@kwdef mutable struct Goblin <: Creature
    _common::_Common = _Common(88, 77)
    weapon::String = "dagger"
end

Base.@kwdef mutable struct Slime <: Creature
    _common::_Common = _Common(55, 44)
    sp::String = "acid"
end

for T in subtypes(Creature)
    n = length(fieldnames(_Common))
    @eval $(nameof(T))(a...) = $T(_Common(a[1:$n]...), a[$n+1:end]...)
    @eval Base.getproperty(x::$T, p::Symbol) =
        p ∈ fieldnames(_Common) ? getfield(x._common, p) : getfield(x, p)
    @eval Base.setproperty!(x::$T, p::Symbol, v) =
        p ∈ fieldnames(_Common) ? setfield!(x._common, p, v) : setfield!(x, p, v)
    @eval Base.propertynames(x::$T) = (fieldnames(_Common)...,
        (p for p in fieldnames($T) if string(p)[1] != '_')...)
    @eval function Base.show(io::IO, x::$T)
        props = getproperty.(Ref(x), propertynames(x))
        print(io, string(nameof($T)), '(', repr(first(props)))
        for p in props[2:end] print(io, ", ", repr(p)) end
        print(io, ')')
    end
end

end

@show Q.Goblin()
@show Q.Slime()
println()
@show g = Q.Goblin(123, 45, "dirty dagger")
@show g.hp
@show g.atk
@show g.weapon
@show g.hp = 99
@show g
@show g.atk = 11
@show g
@show g.weapon = "nothing"
@show g
println()
@show s = Q.Slime(123, 45, "weak acid")
@show s.hp
@show s.atk
@show s.sp
@show s.hp = 66
@show s
@show s.atk = 99
@show s
@show s.sp = "strong acid"
@show s;

Q.Goblin() = Goblin(88, 77, "dagger")
Q.Slime() = Slime(55, 44, "acid")

g = Q.Goblin(123, 45, "dirty dagger") = Goblin(123, 45, "dirty dagger")
g.hp = 123
g.atk = 45
g.weapon = "dirty dagger"
g.hp = 99 = 99
g = Goblin(99, 45, "dirty dagger")
g.atk = 11 = 11
g = Goblin(99, 11, "dirty dagger")
g.weapon = "nothing" = "nothing"
g = Goblin(99, 11, "nothing")

s = Q.Slime(123, 45, "weak acid") = Slime(123, 45, "weak acid")
s.hp = 123
s.atk = 45
s.sp = "weak acid"
s.hp = 66 = 66
s = Slime(66, 45, "weak acid")
s.atk = 99 = 99
s = Slime(66, 99, "weak acid")
s.sp = "strong acid" = "strong acid"
s = Slime(66, 99, "strong acid")


In [2]:
module O

using InteractiveUtils: subtypes

abstract type Creature end

Base.@kwdef mutable struct _Common
    hp::Int = 200
    atk::Int = 100
end
for p in fieldnames(_Common)
    @eval $p(x::Creature) = getfield(x._common, Symbol($p))
    @eval $(Symbol(:set_, p, :!))(x::Creature, v) = setfield!(x._common, Symbol($p), v)
end

Base.@kwdef mutable struct Goblin <: Creature
    _common::_Common = _Common(88, 77)
    weapon::String = "dagger"
end

Base.@kwdef mutable struct Slime <: Creature
    _common::_Common = _Common(55, 44)
    sp::String = "acid"
end

for T in subtypes(Creature)
    n = length(fieldnames(_Common))
    @eval $(nameof(T))(a...) = $T(_Common(a[1:$n]...), a[$n+1:end]...)
    for p in fieldnames(T)
        first(string(p)) == '_' && continue
        @eval $p(x::$T) = getfield(x, Symbol($p))
        @eval $(Symbol(:set_, p, :!))(x::$T, v) = setfield!(x, Symbol($p), v)
    end
end

end

@show O.Goblin()
@show O.Slime()
println()
@show g = O.Goblin(123, 45, "dirty dagger")
@show O.hp(g)
@show O.atk(g)
@show O.weapon(g)
@show O.set_hp!(g, 99)
@show g
@show O.set_atk!(g, 11)
@show g
@show O.set_weapon!(g, "nothing")
@show g
println()
@show s = O.Slime(123, 45, "weak acid")
@show O.hp(s)
@show O.atk(s)
@show O.sp(s)
@show O.set_hp!(s, 66)
@show s
@show O.set_atk!(s, 99)
@show s
@show O.set_sp!(s, "strong acid")
@show s;

O.Goblin() = Main.O.Goblin(Main.O._Common(88, 77), "dagger")
O.Slime() = Main.O.Slime(Main.O._Common(55, 44), "acid")

g = O.Goblin(123, 45, "dirty dagger") = Main.O.Goblin(Main.O._Common(123, 45), "dirty dagger")
O.hp(g) = 123
O.atk(g) = 45
O.weapon(g) = "dirty dagger"
O.set_hp!(g, 99) = 99
g = Main.O.Goblin(Main.O._Common(99, 45), "dirty dagger")
O.set_atk!(g, 11) = 11
g = Main.O.Goblin(Main.O._Common(99, 11), "dirty dagger")
O.set_weapon!(g, "nothing") = "nothing"
g = Main.O.Goblin(Main.O._Common(99, 11), "nothing")

s = O.Slime(123, 45, "weak acid") = Main.O.Slime(Main.O._Common(123, 45), "weak acid")
O.hp(s) = 123
O.atk(s) = 45
O.sp(s) = "weak acid"
O.set_hp!(s, 66) = 66
s = Main.O.Slime(Main.O._Common(66, 45), "weak acid")
O.set_atk!(s, 99) = 99
s = Main.O.Slime(Main.O._Common(66, 99), "weak acid")
O.set_sp!(s, "strong acid") = "strong acid"
s = Main.O.Slime(Main.O._Common(66, 99), "strong acid")
